In [146]:
import contextily as ctx
import folium
import geopandas as gpd
import matplotlib as mpl
import numpy as np
import pandas as pd
import requests

from folium.plugins import GroupedLayerControl


# Carte des quartiers de Marseille

In [147]:
# Importation des données géographiques des quartiers de Marseille
url = 'https://www.data.gouv.fr/api/1/datasets/r/8a8f7f54-7f91-482c-a78c-dd09d893d1b6'
file = requests.get(url)
data = file.content
quartiers_data = gpd.read_file(data)
quartiers_map = quartiers_data.rename({'NOM_QUA' : 'Quartier'}, axis = 1)

# Préparation du gdf d'adresses

In [148]:
# liens de téléchargement pour les 16 arrondissements
url_data = [
    'https://www.data.gouv.fr/api/1/datasets/r/84b5a6c5-5c0f-4518-8f6e-6e00b5fcacda',
    'https://www.data.gouv.fr/api/1/datasets/r/b5f2a20b-5951-4698-9b11-b86c51940eab',
    'https://www.data.gouv.fr/api/1/datasets/r/39aedda3-9479-40d5-97cf-405374813e09',
    'https://www.data.gouv.fr/api/1/datasets/r/c7126419-50d2-4eeb-9f9f-cca76e0ba4cf',
    'https://www.data.gouv.fr/api/1/datasets/r/15f80a80-34a2-47b7-80b8-fed1cde87f64',
    'https://www.data.gouv.fr/api/1/datasets/r/2b657228-e856-4b16-bd6e-d6fbeaf0b644',
    'https://www.data.gouv.fr/api/1/datasets/r/7659198f-da95-46d3-918a-f1c89db62131',
    'https://www.data.gouv.fr/api/1/datasets/r/04a47775-3b5a-4ed7-9d17-6dcc17adf47a',
    'https://www.data.gouv.fr/api/1/datasets/r/46c4de02-f529-4a42-8f24-cdc12e55e35c',
    'https://www.data.gouv.fr/api/1/datasets/r/699d4429-dd09-4978-adc8-78afab7aecc7',
    'https://www.data.gouv.fr/api/1/datasets/r/364bebd5-6762-493d-bb9a-a25d4e11edf1',
    'https://www.data.gouv.fr/api/1/datasets/r/ea5017ad-5e8b-4186-b560-07ea77960c94',
    'https://www.data.gouv.fr/api/1/datasets/r/c7024730-396d-4f0a-a238-e638fffade11',
    'https://www.data.gouv.fr/api/1/datasets/r/b0fefb17-69aa-45f2-b09e-0c29cb2cd689',
    'https://www.data.gouv.fr/api/1/datasets/r/da06caf0-2cc6-4282-82c6-e8da1a57d3ed',
    'https://www.data.gouv.fr/api/1/datasets/r/ee39eb43-74f7-490c-b728-2fea9d34dfeb'
    ]


In [149]:
# initialisation du df d'adresses avec le premier arrondissement
adresses = pd.read_csv(url_data[0], sep = ';')

# boucle de concaténation des autres attondissements
for url in url_data[1:]:
    data = pd.read_csv(url, sep = ";")
    adresses = pd.concat([adresses, data], axis = 0, ignore_index = True)

adresses

# À noter : variables 'x' et 'y' dans le système Lambert93 / variables 'long' et 'lat' dans le système WGS 

,uid_adresse,cle_interop,commune_insee,commune_nom,commune_deleguee_insee,commune_deleguee_nom,voie_nom,lieudit_complement_nom,numero,suffixe,position,x,y,long,lat,cad_parcelles,source,date_der_maj,certification_commune
0,90259,13201_2838_00002,13201,1er arrondissement,NaN,NaN,Rue docteur claudius regaud,belsunce,2,NaN,entrée,892909.23,6247635.35,5.376411,43.301756,131201801 A0355,commune,2022-11-24,1
1,75934,13201_0056_00020,13201,1er arrondissement,NaN,NaN,Rue de l' academie,noailles,20,NaN,entrée,893278.13,6246964.78,5.380704,43.295626,131201803 B0107,commune,2022-11-24,1
2,75936,13201_0056_00024,13201,1er arrondissement,NaN,NaN,Rue de l' academie,noailles,24,NaN,entrée,893289.00,6246979.84,5.380844,43.295759,131201803 B0110,commune,2022-11-24,1
3,89905,13201_0056_00001,13201,1er arrondissement,NaN,NaN,Rue de l' academie,noailles,1,NaN,entrée,893225.91,6246907.13,5.380040,43.295122,131201803 A0237,commune,2022-11-24,1
4,75937,13201_0056_00026,13201,1er arrondissement,NaN,NaN,Rue de l' academie,noailles,26,NaN,entrée,893294.50,6246987.48,5.380914,43.295826,131201803 B0111,commune,2022-11-24,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91640,13831,13216_4691_00001,13216,16e arrondissement,NaN,NaN,Rue jean jacques vernazza,l'estaque,1,NaN,entrée,888369.33,6254229.65,5.322907,43.362266,131216908 E0205,commune,2022-11-24,1
91641,107641,13216_z436_00011,13216,16e arrondissement,NaN,NaN,Rue kader tighilt,l'estaque,11,NaN,entrée,888244.97,6254142.45,5.321343,43.361515,131216908 E0459,commune,2022-11-24,1
91642,84948,13216_0774_00059,13216,16e arrondissement,NaN,NaN,Traverse de la barre,saint andre,59,NaN,entrée,890202.73,6254707.15,5.345678,43.366072,131216910 A0067,commune,2022-11-24,1
91643,84949,13216_0774_00049,13216,16e arrondissement,NaN,NaN,Traverse de la barre,saint andre,49,NaN,entrée,890208.32,6254653.90,5.345727,43.365592,131216910 A0056,commune,2022-11-24,1


In [150]:
adresses = adresses[['commune_insee', 'commune_nom', 'lieudit_complement_nom', 'long', 'lat']]
adresses = adresses.rename({'lieudit_complement_nom' : 'Quartier'}, axis = 1)

In [151]:
# Conversion en geodataframe
gdf_adresses = gpd.GeoDataFrame(
    adresses,
    geometry=gpd.points_from_xy(
        adresses['long'],
        adresses['lat'],
        crs="EPSG:4326"))


# Préparation du gdf de pav

In [152]:
# Importation des données concernant les points d'apport volontaire
url = 'https://www.data.gouv.fr/api/1/datasets/r/e46c6879-49e7-4727-8f3b-62df90ac5a5a'
gouv_data = pd.read_csv(url, sep = ';')

In [153]:
# Extraction des variables d'intérêt
pav_data = gouv_data[['OBJECTID', 'Code Commune INSEE', 'Quartier', 'Volume intérieur m3', 'longitude', 'latitude', 'Type de flux']]

# Extraction des observations concernant les quartiers de Marseille
pav = pav_data.loc[(pav_data['Code Commune INSEE'] > 13200) & (pav_data['Code Commune INSEE'] < 13217)]
pav = pav.dropna(axis = 0, subset = ['Quartier'])


# conversion en geodataframe
gdf_pav = gpd.GeoDataFrame(
    pav,
    geometry=gpd.points_from_xy(
        pav['longitude'],
        pav['latitude'],
        crs="EPSG:4326"))


In [154]:
# Le format .shp limite à 10 caractères le nom des variables
# On remplace les types de flux par des abréviations pour faciliter
# le nommage de futures variables


types_dict = {
    'Verre' : 'V',
    'Biflux' : 'Bf',
    'OM' : 'OM',
    'Biodechets' : 'Bio',
    'Textile' : 'T',
    'Papier / Journaux' : 'P',
    'Emballage' : 'E',
    'Carton' : 'C'
    }



gdf_pav['Type de flux'] = gdf_pav['Type de flux'].replace(types_dict)


# Uniformisation des noms de quartier

In [155]:
gdf_adresses['Quartier'] = gdf_adresses['Quartier'].apply(lambda x : x.upper())

In [156]:
# Correction des noms de quartiers 

gdf_pav['Quartier'] = gdf_pav['Quartier'].replace(
    {
    'LE ROUCAS' : 'LE ROUCAS BLANC',
    'ST BARNANE' : 'SAINT BARNABE',
    'ROUCAS BLANC' : 'LE ROUCAS BLANC'
    }
)

quartiers_map['Quartier'] = quartiers_map['Quartier'].replace(
    {
        'SAINT MAURON' : 'SAINT MAURONT',
        'VIELLE CHAPELLE' : 'VIEILLE CHAPELLE',
        'GRANDS CARMES' : 'LES GRANDS CARMES',
        'LA VILETTE' : 'LA VILLETTE',
        'CHUTES LAVIE' : 'LES CHUTES LAVIES',
        'CINQ AVENUES' : 'LES CINQ AVENUES',
        'ROUCAS BLANC' : 'LE ROUCAS BLANC',
        'POINTE ROUGE' : 'LA POINTE ROUGE',
        'SAINTE MARGUERITE' : 'STE MARGUERITE',
        'CHATEAU-GOMBERT' : 'CHATEAU GOMBERT'
        

    }
)



# Recherche des pav les plus proches de chaque adresse

In [157]:
gdf_adresses = gdf_adresses.to_crs(2154)
gdf_pav = gdf_pav.to_crs(2154)

In [158]:
# calcul de la distance du pav le plus proche d'une adresse donnée
def distances_pav(adresse, type_pav):
    distance = 10000
    for pav in gdf_pav.loc[gdf_pav['Type de flux'] == type_pav]['geometry']:
        if adresse.distance(pav) < distance:
            distance = adresse.distance(pav)
    return round(distance)



# Chargement du fichier traité / ou traitement du fichier 
try :
    quartiers_dist = gpd.read_file('../data/cache/distances_moyenne_adresses_pav.shp')
    
except :  
    print('Mean distances file not found, generating new file')

    for type in types_dict.values():
        # ajout, dans le gdf d'adresses, des distances du pav le plus proche pour chaque type de flux
        gdf_adresses['dist_' + type] = gdf_adresses['geometry'].apply(lambda x : distances_pav(x, type))
        
    # groupby par quartier avec distance moyenne
    gdf_distance_moyenne = gdf_adresses.dissolve(by = 'Quartier', aggfunc = 'mean', numeric_only = True)
        
    # jointure avec les données des quartiers
    quartiers_dist = quartiers_map.merge(gdf_distance_moyenne.drop(['geometry'], axis = 1), on = 'Quartier', how = 'left')
    
    # save
    quartiers_dist.to_file('../data/cache/distances_moyenne_adresses_pav.shp', index = False)



In [159]:
quartiers_dist

,DEPCO,NOM_CO,Quartier,commune_in,long,lat,dist_V,dist_Bf,dist_OM,dist_Bio,dist_T,dist_P,dist_E,dist_C,geometry
0,13201,Marseille 1er Arrondissemen,BELSUNCE,13201.0,5.377537,43.299420,76.773424,76.874787,121.103918,70.418228,215.602215,10000.0,10000.0,146.987223,"POLYGON ((5.38086 43.29924, 5.38087 43.29912, ..."
1,13201,Marseille 1er Arrondissemen,CHAPITRE,13201.0,5.384295,43.300697,55.880923,55.091130,115.856622,58.772783,124.122722,10000.0,10000.0,198.136087,"POLYGON ((5.38525 43.29906, 5.38485 43.29889, ..."
2,13201,Marseille 1er Arrondissemen,NOAILLES,13201.0,5.379722,43.294908,73.810967,77.089466,96.383838,73.522367,220.571429,10000.0,10000.0,128.261183,"POLYGON ((5.3816 43.29573, 5.38177 43.29518, 5..."
3,13201,Marseille 1er Arrondissemen,OPERA,13201.0,5.375369,43.293388,70.259370,64.938531,70.284858,69.304348,198.949025,10000.0,10000.0,85.976012,"POLYGON ((5.37729 43.29222, 5.37663 43.29206, ..."
4,13201,Marseille 1er Arrondissemen,SAINT CHARLES,13201.0,5.390644,43.303009,60.624250,60.552221,244.925570,63.605042,128.368547,10000.0,10000.0,229.115246,"POLYGON ((5.38022 43.30141, 5.38007 43.3014, 5..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,13215,Marseille 15e Arrondissemen,LA VISTE,13215.0,5.358076,43.360432,120.058824,121.937255,113.178431,203.162745,918.419608,10000.0,10000.0,6345.386275,"POLYGON ((5.35771 43.35337, 5.35756 43.35336, ..."
107,13216,Marseille 16e Arrondissemen,L'ESTAQUE,13216.0,5.315608,43.363459,144.975744,149.800441,344.675303,207.256340,409.406836,10000.0,10000.0,7840.980706,"POLYGON ((5.32691 43.36112, 5.32694 43.36111, ..."
108,13216,Marseille 16e Arrondissemen,LES RIAUX,13216.0,5.305061,43.364515,245.991453,245.991453,715.606838,278.384615,885.358974,10000.0,10000.0,8403.589744,"POLYGON ((5.30862 43.36508, 5.30878 43.36501, ..."
109,13216,Marseille 16e Arrondissemen,SAINT ANDRE,13216.0,5.342328,43.355652,171.803862,202.806911,345.753049,168.191057,190.136179,10000.0,10000.0,6109.652439,"MULTIPOLYGON (((5.33997 43.34367, 5.33992 43.3..."


In [160]:
# arrondi des distances à l'entier
for type in types_dict.values():
    dist_type = 'dist_' + str(type)
    quartiers_dist[dist_type] = quartiers_dist[dist_type].apply(lambda x : round(x) if pd.notnull(x) else np.nan)

In [161]:
quartiers_dist.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 111 entries, 0 to 110
Data columns (total 15 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   DEPCO       111 non-null    object  
 1   NOM_CO      111 non-null    object  
 2   Quartier    111 non-null    object  
 3   commune_in  110 non-null    float64 
 4   long        110 non-null    float64 
 5   lat         110 non-null    float64 
 6   dist_V      110 non-null    float64 
 7   dist_Bf     110 non-null    float64 
 8   dist_OM     110 non-null    float64 
 9   dist_Bio    110 non-null    float64 
 10  dist_T      110 non-null    float64 
 11  dist_P      110 non-null    float64 
 12  dist_E      110 non-null    float64 
 13  dist_C      110 non-null    float64 
 14  geometry    111 non-null    geometry
dtypes: float64(11), geometry(1), object(3)
memory usage: 13.1+ KB


In [162]:
# choix des classes d'intervalles
ref_bins = [0, 50, 100, 150, 200, 300, 500]
ref_labels = ['< 50', '(50, 100]', '(100, 150]', '(150, 200]', '(200, 300]', '(300, 500]', '> 500']

# création d'un dictionnaire de couleurs pour chaque classe d'intervalle à partir de la cmap 'Oranges'
n_bins = len(ref_bins)
cmap = mpl.colormaps['Oranges']
ref_colors = cmap(np.linspace(0, 1, n_bins))
custom_cmap = mpl.colors.ListedColormap(ref_colors)


color_dict = {}
color_dict[np.nan] = '#808080'
for n in range(n_bins):
    color_dict[ref_labels[n]] = mpl.colors.rgb2hex(custom_cmap(n))

In [163]:
# on ne garde que les bins et les labels nécessaires pour chaque colonne
# cut selon les bins retenus
# assignation des couleurs pour chaque classe
for type in types_dict.values():
    dist_type = 'dist_' + str(type)
    moy_max = round(max(quartiers_dist[dist_type]))
    bins = [bin for bin in ref_bins if bin < moy_max]
    bins.append(moy_max)
    labels = ref_labels[:len(bins)-1]
    quartiers_dist[dist_type + '_cat'] = pd.cut(quartiers_dist[dist_type], bins = bins, labels = ref_labels)
    quartiers_dist[dist_type + '_col'] = quartiers_dist[dist_type+'_cat'].map(color_dict)

# Génération de la carte

In [164]:
quartiers_dist = quartiers_dist.to_crs(3857)
gdf_pav = gdf_pav.to_crs(3857)

In [170]:
def tooltip_label(x) : 
    if x == 10000.0:
        x = '>=10000'
    else :
        x = str(x)[:-2]
    return x


for type in types_dict.values():
    quartiers_dist['dist_'+type] = quartiers_dist['dist_'+type].apply(lambda x : tooltip_label(x))

In [171]:
# fond de carte
map = folium.Map(
    location = [43.275, 5.4],
    tiles = 'cartodbpositron',
    zoom_start = 12,
    max_zoom=17,
    control_scale=True
    )


# ajout des couches pour chaque type de flux
layers = {}

for n,type in enumerate(types_dict.values()):
    layer_name = 'fg_'+ type 
    label = list(types_dict.keys())[n]
    layers[layer_name] = folium.FeatureGroup(name = label).add_to(map)

    gdf_pav.loc[gdf_pav['Type de flux']==type].explore(
        m = layers[layer_name],
        marker_kwds=dict(radius=0.5, fill=True),
        color = 'Gray'
    )


    quartiers_dist.explore(
        m = layers[layer_name],
        color = 'dist_'+type+'_col',
        tooltip = ['Quartier', 'dist_'+type],
        style_kwds = {'color' : 'Black'}
    )

    
    map.add_child(layers[layer_name])


# Contrôle de l'affichage des couches
GroupedLayerControl(
    groups={'Type de flux': [layer for layer in layers.values()]},
    collapsed=False,
).add_to(map)



map_title = "Distance moyenne au pav le plus proche par quartier"
title_html = f'<h1 align="center" style="font-size:24px" >{map_title}</h1>'
map.get_root().html.add_child(folium.Element(title_html))


map.save('../cartes/map_distance_moyenne.html')


/tmp/ipykernel_4668/876797860.py:19: UserWarning: The GeoSeries you are attempting to plot is composed of empty geometries. Nothing has been displayed.
  gdf_pav.loc[gdf_pav['Type de flux']==type].explore(
/tmp/ipykernel_4668/876797860.py:19: UserWarning: The GeoSeries you are attempting to plot is composed of empty geometries. Nothing has been displayed.
  gdf_pav.loc[gdf_pav['Type de flux']==type].explore(
